# Load Answers and Answer Sources to Database

This notebook loads:
1. `4_answer_gen.csv` into the `faq_answers` table
2. `4_answer_source_gen.csv` into the `faq_answer_sources` table

In [11]:
import pandas as pd
from pathlib import Path
from database.backends.factory import BackendFactory
from database.config import DatabaseConfig
from database.repository import FAQRepository

In [12]:
# Define file paths
answers_csv = Path('data_ingestion/data/4_answer_gen.csv')
answer_sources_csv = Path('data_ingestion/data/4_answer_source_gen.csv')
db_path = Path('databases/faq_update.db')

# Create database configuration
config = DatabaseConfig(backend='sqlite', db_path=str(db_path))
print(f"Database: {db_path}")

Database: databases\faq_update.db


In [13]:
# Load CSV files
print("Loading answers CSV...")
df_answers = pd.read_csv(answers_csv)
print(f"Loaded {len(df_answers)} answers")
print("\nAnswers columns:", df_answers.columns.tolist())
print("\nFirst few rows:")
df_answers.head()

Loading answers CSV...
Loaded 74 answers

Answers columns: ['answer_id', 'question_id', 'answer_text', 'answer_format', 'confidence_score', 'status', 'created_at', 'modified_at']

First few rows:


answer_id  \
0  3cfd955373c5a47d686687cc2bfdbb315fd5e18426a92e...   
1  40f925e47a0d41009a21a82a92978373cb59d2d8383650...   
2  604b1830161ceb0b429154f60fcdcf695bb21f76bfab09...   
3  5d5916b07206148742849f84f96790633daf34f203476a...   
4  7e8b2aee9fe2245b06f7db6c9a0ac7978309bf9085bb66...   

                                         question_id  \
0  0a273c5f40c27b21443b02a67699c9194980f451bf5c88...   
1  0d08e2eab477c089cf592b1be153ff890bbe3a35b1d290...   
2  1456f7fb640f4183fa660841c708911be3c30f86752a1e...   
3  1574ae5e08917dd26acbd907e660ecd1346e1c6498d852...   
4  171e88489d4a5f5d2fec88f8d56c6af5a7b87ee4383ed2...   

                                         answer_text answer_format  \
0  Employees can access their pay stubs through t...          html   
1  Remote employees must be available during core...          html   
2  Yes, company email is subject to monitoring an...          html   
3  I do not have enough information to answer thi...          html   
4  The password requirements for employees are as...          html   

   confidence_score  status                created_at  \
0               NaN  active  2025-10-29T19:33:39.141Z   
1               NaN  active  2025-10-29T19:33:39.141Z   
2               NaN  active  2025-10-29T19:33:39.141Z   
3               NaN  active  2025-10-29T19:33:39.141Z   
4               NaN  active  2025-10-29T19:33:39.141Z   

                modified_at  
0  2025-10-29T19:33:39.142Z  
1  2025-10-29T19:33:39.142Z  
2  2025-10-29T19:33:39.142Z  
3  2025-10-29T19:33:39.142Z  
4  2025-10-29T19:33:39.142Z

In [14]:
print("Loading answer sources CSV...")
df_answer_sources = pd.read_csv(answer_sources_csv)
print(f"Loaded {len(df_answer_sources)} answer sources")
print("\nAnswer sources columns:", df_answer_sources.columns.tolist())
print("\nFirst few rows:")
df_answer_sources.head()

Loading answer sources CSV...
Loaded 74 answer sources

Answer sources columns: ['source_id', 'answer_id', 'content_checksum', 'is_primary_source', 'contribution_weight', 'context_employed', 'is_valid', 'valid_from', 'valid_until', 'invalidation_reason', 'invalidated_by_change_id', 'created_at']

First few rows:


source_id                                          answer_id  \
0          1  3cfd955373c5a47d686687cc2bfdbb315fd5e18426a92e...   
1          2  40f925e47a0d41009a21a82a92978373cb59d2d8383650...   
2          3  604b1830161ceb0b429154f60fcdcf695bb21f76bfab09...   
3          4  5d5916b07206148742849f84f96790633daf34f203476a...   
4          5  7e8b2aee9fe2245b06f7db6c9a0ac7978309bf9085bb66...   

                                    content_checksum  is_primary_source  \
0  32cbadc16b6c3e6976b4158e3ffd945855495cb3ccc17d...               True   
1  525a301d7750b939ac2adf0e9f3690770b81277fd5dcc8...               True   
2  26b8fffa2bb44d9ed01d6878fa75f7efc7c3d6355478e3...               True   
3  26b8fffa2bb44d9ed01d6878fa75f7efc7c3d6355478e3...               True   
4  21f1d25d4f2dc7ce53d72cc050c32738362e1e12536d0b...               True   

   contribution_weight                                   context_employed  \
0                    1  # Payroll Processing Guide - Tax Withholding\n...   
1                    1  # Remote Work Policy - Eligibility and Guideli...   
2                    1  # Email Policy\n\nCompany email should be used...   
3                    1  # Email Policy\n\nCompany email should be used...   
4                    1  # IT Security Policy - Password Requirements\n...   

   is_valid                valid_from  valid_until  invalidation_reason  \
0      True  2025-10-29T19:33:43.736Z          NaN                  NaN   
1      True  2025-10-29T19:33:43.736Z          NaN                  NaN   
2      True  2025-10-29T19:33:43.736Z          NaN                  NaN   
3      True  2025-10-29T19:33:43.736Z          NaN                  NaN   
4      True  2025-10-29T19:33:43.736Z          NaN                  NaN   

   invalidated_by_change_id                created_at  
0                       NaN  2025-10-29T19:33:43.737Z  
1                       NaN  2025-10-29T19:33:43.737Z  
2                       NaN  2025-10-29T19:33:43.737Z  
3                       NaN  2025-10-29T19:33:43.737Z  
4                       NaN  2025-10-29T19:33:43.737Z

In [15]:
# Create backend and repository
print("=" * 70)
print("INITIALIZING FAQ REPOSITORY - Repository Pattern")
print("=" * 70)
print()

backend = BackendFactory.create_backend(config)
faq_repo = FAQRepository(backend)

print(f"✓ Backend created: {backend.__class__.__name__}")
print(f"✓ Repository created: FAQRepository")
print()

2025-11-02 14:42:17.134 | DEBUG    | database.backends.factory:create_backend:116 - Returning existing backend singleton


INITIALIZING FAQ REPOSITORY - Repository Pattern

✓ Backend created: SQLiteBackend
✓ Repository created: FAQRepository



In [16]:
# Save answers to faq_answers table using FAQRepository
print("=" * 70)
print("INGESTING ANSWERS - Repository Pattern")
print("=" * 70)
print()

print("📊 Before ingestion:")
stats_before = faq_repo.get_faq_stats()
print(f"   Total answers: {stats_before.get('total_answers', 0)}")
print()

print("💾 Ingesting answers using FAQRepository...")
try:
    result = faq_repo.ingest_answers(
        answers_df=df_answers,
        validate=True,
        validate_fk=True,  # Validate that question_ids exist
        if_exists='append'
    )
    
    print(f"✅ SUCCESS")
    print(f"   Rows inserted: {result.get('rows_inserted', 0)}")
    print(f"   Message: {result.get('message', 'Ingestion completed')}")
except Exception as e:
    print(f"❌ FAILED")
    print(f"   Error: {str(e)}")
    raise

print()

# Verify
stats_after = faq_repo.get_faq_stats()
print("📊 After ingestion:")
print(f"   Total answers: {stats_after.get('total_answers', 0)}")
print()
print("=" * 70)

INGESTING ANSWERS - Repository Pattern

📊 Before ingestion:


2025-11-02 14:42:17.161 | DEBUG    | database.repository.faq_repository:get_faq_stats:1109 - Calculating FAQ statistics
2025-11-02 14:42:17.167 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:17.167 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:17.168 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:17.169 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:17.170 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:17.172 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:17.173 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:17.176 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42

   Total answers: 0

💾 Ingesting answers using FAQRepository...
✅ SUCCESS
   Rows inserted: 74
   Message: Successfully inserted 74 rows into faq_answers

📊 After ingestion:
   Total answers: 74



In [17]:
# Save answer sources to faq_answer_sources table using FAQRepository
print()
print("=" * 70)
print("INGESTING ANSWER SOURCES - Repository Pattern")
print("=" * 70)
print()

print("💾 Ingesting answer sources using FAQRepository...")
try:
    # Remove 'source_id' column if present (auto-generated by database)
    if 'source_id' in df_answer_sources.columns:
        df_to_ingest = df_answer_sources.drop(columns=['source_id'])
    else:
        df_to_ingest = df_answer_sources
    
    result = faq_repo.ingest_answer_sources(
        sources_df=df_to_ingest,
        validate=True,
        validate_fk=True  # Validate that answer_ids exist
    )
    
    print(f"✅ SUCCESS")
    print(f"   Rows inserted: {result.get('rows_inserted', 0)}")
    print(f"   Message: {result.get('message', 'Ingestion completed')}")
except Exception as e:
    print(f"❌ FAILED")
    print(f"   Error: {str(e)}")
    raise

print()
print("=" * 70)
print("✅ INGESTION COMPLETE")
print("=" * 70)

2025-11-02 14:42:17.240 | INFO     | database.repository.faq_repository:ingest_answer_sources:1996 - Ingesting 74 answer sources in bulk
2025-11-02 14:42:17.245 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 74 rows
2025-11-02 14:42:17.246 | DEBUG    | database.repository.base:execute_query:590 - Query returned 74 rows
2025-11-02 14:42:17.251 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:17.251 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:17.255 | INFO     | database.backends.sqlite.backend:ingest_dataframe:426 - Inserted 74 rows into faq_answer_sources
2025-11-02 14:42:17.256 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:17.257 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully
2025-11-02 14:42:17.258 | INFO     | database.repository.faq_repository:inges


INGESTING ANSWER SOURCES - Repository Pattern

💾 Ingesting answer sources using FAQRepository...
✅ SUCCESS
   Rows inserted: 74
   Message: Successfully inserted 74 rows into faq_answer_sources

✅ INGESTION COMPLETE


In [18]:
# Display sample data from database using Repository
print()
print("=" * 70)
print("VERIFICATION - Repository Pattern")
print("=" * 70)
print()

print("📋 Sample answers from repository:")
# Use repository method instead of direct backend query
sample_answers = faq_repo.list_answers(limit=5, order_by="created_at DESC")
df_sample_answers = pd.DataFrame(sample_answers)
print(df_sample_answers)
print()

2025-11-02 14:42:17.287 | DEBUG    | database.repository.faq_repository:list_answers:456 - Listing answers (question_id=None, status=None, format=None, min_confidence=None, limit=5)
2025-11-02 14:42:17.294 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 5 rows
2025-11-02 14:42:17.295 | DEBUG    | database.repository.base:execute_query:590 - Query returned 5 rows
2025-11-02 14:42:17.296 | DEBUG    | database.repository.faq_repository:list_answers:501 - Found 5 answers



VERIFICATION - Repository Pattern

📋 Sample answers from repository:
                                           answer_id  \
0  b8f8f776f1f6ae1fc1a77279dab55ba9becdb235c0abba...   
1  423c2a4e8a47c5dc4d44fa677b144964908ffc93491a75...   
2  53cb6aab161d21f0b73e7992a7707436697dc6871d2840...   
3  ffcf32e5a93677c32a752f1015278ec3a8f5c8c6d66b20...   
4  76d21d8ae6666cc2853d98adabee295f519021598d4214...   

                                         question_id  \
0  ffb961d88fa923b1c256f3f7c3562719b4d8c16ef54d52...   
1  feae93a3b8aa895bf5f878108a5369cb9b32395b4e87bd...   
2  fc6bf0ff097412a9ef8b6c3d9aa1cfa3be03de0adb50ea...   
3  fb6b366129f92d88d36b190d44d489ff7ade64eeba0df5...   
4  fa850b711de5ee9ec91d519e9d6ee2c21d0b013481d6e6...   

                                         answer_text answer_format  \
0  Vacation time is accrued based on tenure as fo...          html   
1  Federal, state, and local taxes are automatica...          html   
2  Yes, employees may add dependents to their 

In [19]:
print("📋 Sample answer sources from repository:")
# Use repository method instead of direct backend query
sample_sources = faq_repo.list_answer_sources(limit=5, order_by="created_at DESC")
df_sample_sources = pd.DataFrame(sample_sources)
print(df_sample_sources)
print()

# Get overall stats
stats = faq_repo.get_faq_stats()
print("📊 Repository Statistics:")
print(f"   Total questions: {stats.get('total_questions', 0)}")
print(f"   Total answers: {stats.get('total_answers', 0)}")
print(f"   Answers by status: {stats.get('answers_by_status', {})}")
print(f"   Questions without answers: {stats.get('questions_without_answers', 0)}")
print(f"   Average confidence score: {stats.get('avg_confidence_score', 'N/A')}")
print()

print("=" * 70)
print("✅ VERIFICATION COMPLETE")
print("=" * 70)

2025-11-02 14:42:17.321 | DEBUG    | database.repository.faq_repository:list_answer_sources:1897 - Listing answer sources (answer_id=None, is_valid=None, is_primary=None, limit=5)
2025-11-02 14:42:17.323 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 5 rows
2025-11-02 14:42:17.324 | DEBUG    | database.repository.base:execute_query:590 - Query returned 5 rows
2025-11-02 14:42:17.327 | DEBUG    | database.repository.faq_repository:list_answer_sources:1938 - Found 5 answer sources
2025-11-02 14:42:17.331 | DEBUG    | database.repository.faq_repository:get_faq_stats:1109 - Calculating FAQ statistics
2025-11-02 14:42:17.333 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:17.334 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:17.336 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:17.336 | DEBUG   

📋 Sample answer sources from repository:
   source_id                                          answer_id  \
0          1  3cfd955373c5a47d686687cc2bfdbb315fd5e18426a92e...   
1          2  40f925e47a0d41009a21a82a92978373cb59d2d8383650...   
2          3  604b1830161ceb0b429154f60fcdcf695bb21f76bfab09...   
3          4  5d5916b07206148742849f84f96790633daf34f203476a...   
4          5  7e8b2aee9fe2245b06f7db6c9a0ac7978309bf9085bb66...   

                                    content_checksum  is_primary_source  \
0  32cbadc16b6c3e6976b4158e3ffd945855495cb3ccc17d...                  1   
1  525a301d7750b939ac2adf0e9f3690770b81277fd5dcc8...                  1   
2  26b8fffa2bb44d9ed01d6878fa75f7efc7c3d6355478e3...                  1   
3  26b8fffa2bb44d9ed01d6878fa75f7efc7c3d6355478e3...                  1   
4  21f1d25d4f2dc7ce53d72cc050c32738362e1e12536d0b...                  1   

   contribution_weight                                   context_employed  \
0                  1.0  # Pa

2025-11-02 14:42:17.341 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:17.343 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:17.344 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:17.345 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:17.346 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:17.347 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:17.349 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:17.350 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:17.351 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:17.352 | DEBUG 

📊 Repository Statistics:
   Total questions: 74
   Total answers: 74
   Answers by status: {'active': 74}
   Questions without answers: 0
   Average confidence score: None

✅ VERIFICATION COMPLETE


In [20]:
# No need to manually close connection - repository pattern handles this
print()
print("✅ Process complete!")


✅ Process complete!
